In [ ]:
#r "D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
// #r "C:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Debug\net5.0\bossspad.dll"

//#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
Init();

## Open Database

In [ ]:
BoSSSshell.WorkflowMgm.Init("CoFloFlame_MFComparison__");

Project name is set to 'CoFloFlame_MFComparison__'.
Creating database '\\hpccluster\hpccluster-scratch\gutierrez\CoFloFlame_MFComparison__'.


In [ ]:
 static var myBatch = GetDefaultQueue();
 (myBatch as MsHPC2012Client).ComputeNodes = new string[]{"HPCLUSTER3"};

static var myDb = BoSSSshell.WorkflowMgm.DefaultDatabase;

BoSSSshell.WorkflowMgm.DefaultDatabase = myDb;

## Case configuration

In [ ]:
int NumberOfProcMF = 8;
int dgMF = 2; // Mass fraction DG degree
int dgFull = 3;
int nCells = 10;
double InitialMassFuelIn = 0.02400; //kg/m2s
double InitialMassAirIn = 0.02400*3 ; //kg/m2s

bool BurkeSchu = true; // Configuration for the BS flame => constant density, uniform velocity inlet, 

## Create grid

In [ ]:
public static class GridFactory {

    public static Grid2D GenerateGrid_FullSystem(int nCells, double r, double delta, double R, double zlength) {
         // Geometry
        // All lenghts are non.dimensionalized with Lref = fuel inlet radius (0.2 cm)

        // double r = 0.635 / 100 * factor; // Radius inner cylinder, m
        // double delta = r / 5;
        // double R = 2.54 / 100 * factor * 3 + delta; // Radius outter cylinder, m
        double LRef = r;

        // double zlength = 20.0 / 100 * 2;// meters
        double xmin = -R / LRef;
        double xmax = +R / LRef;
        double ymin = 0;
        double ymax = zlength / LRef;
        double rAd = r / LRef;
        double deltaNonDim = delta / LRef;

        double leftmidpoint = (xmin - rAd) / 2;
        double rightmidpoint = (xmax + rAd) / 2;

    
        double stretchfactorY = 0.98 * 1;

        double sf1 = 0.97 * 1;
        double sf2 = 0.95 * 1 * 0;
        double sf3 = 0.5;
        int n1 = (int)(2.0 * nCells);
        int n2 = (int)(0.5 * nCells);
        int n3 = (int)(2.0 * nCells);
        int n4 = (int)(0.2* nCells)*0 +1;
        int n5 = (int)(1.0* nCells);
        Console.WriteLine("nCells: " + nCells);
        Console.WriteLine("n4: " + n4);
        //var xNodes1 = GenericBlas.SinLinSpacing(xmin, -rAd, sf1, n1 + 1);
        //var xNodes2 = GenericBlas.SinLinSpacing(-rAd, rAd, sf2, n2 + 1);
        //var xNodes3 = GenericBlas.SinLinSpacing(rAd, xmax, sf3, n3 + 1);
        var wallNodes = (GenericBlas.Linspace(rAd, rAd + deltaNonDim, n4 + 1)).ToArray(); // nodes corresponding to the wall

        List<double> xNodes2 = (GenericBlas.SinLinSpacing(-rAd, rAd, sf2, n2 + 1)).ToList(); // nodes corresponding to the fuel inlet
        List<double> xNodes3 = (GenericBlas.SinLinSpacing(rAd, (xmax - rAd) * 2 + rAd, sf3, n1 * 2 + 1).ToList()); // Nodes corresponding to the oxidizer inlet, right part

        var myXnodes3 = xNodes3.GetSubVector(0, xNodes3.Count / 2 + 1); // Take only "left side" of node array

        for (int i = 0; i < myXnodes3.Count(); i++) { // Move values for accounting the wall width
            myXnodes3[i] = myXnodes3[i] + deltaNonDim;
        }

        var myxNodes1 = myXnodes3.CloneAs();
        myxNodes1.ScaleV(-1.0);
        Array.Reverse(myxNodes1);

        var wallNodesLeft = wallNodes.CloneAs();
        wallNodesLeft.ScaleV(-1.0);
        Array.Reverse(wallNodesLeft);

        List<double> list2 = new List<double>();
        list2.AddRange(myxNodes1.Take(n1 + 0).ToList());
        list2.AddRange(wallNodesLeft.Take(n4 + 0).ToList());
        list2.AddRange(xNodes2.Take(n2 + 0).ToList());
        list2.AddRange(wallNodes.Take(n4 + 0).ToList());
        list2.AddRange(myXnodes3.Take(n3 + 1).ToList());
        double[] _xNodes = list2.ToArray();


        int n6 =  (2 * nCells) * 3 + 1;
        var _yNodes = GenericBlas.SinLinSpacing(ymin, ymax * 2, stretchfactorY, (2 * nCells) * 4 + 1);
        var myYnodes = _yNodes.GetSubVector(0, _yNodes.Length / 2 + 1); // I just want a fine mesh in the bottom part of the grid.


        double tubeHeight = rAd * 4.0;

        // var wallNodesY = (GenericBlas.Linspace(0, tubeHeight, n5 + 1)).ToArray(); // nodes corresponding to the wall
        var wallNodesY = GenericBlas.SinLinSpacing(0, tubeHeight * 2, stretchfactorY, (2 * n5) * 1 + 1);
        var myYnodes1 = wallNodesY.GetSubVector(wallNodesY.Length / 2 ,wallNodesY.Length / 2+1   ); 
        for (int i = 0; i < myYnodes1.Count(); i++) { // Move values for accounting the wall height
            myYnodes1[i] = myYnodes1[i] - tubeHeight;
        }

        for (int i = 0; i < myYnodes.Count(); i++) { // Move values for accounting the wall height
            myYnodes[i] = myYnodes[i] + tubeHeight;
        }

        List<double> listY = new List<double>();
        listY.AddRange(myYnodes1.Take(n5 + 0).ToList());
        listY.AddRange(myYnodes.Take(n6/2 + 1).ToList());
       
        double[] _yNodes3 = listY.ToArray();
        Console.WriteLine("rAd:" + rAd);
        Console.WriteLine("tubeHeight:" + tubeHeight);
        double[] CutOut1Point1 = new double[2] { rAd, 0.0 };
        double[] CutOut1Point2 = new double[2] { rAd + deltaNonDim , tubeHeight};

        double[] CutOut2Point1 = new double[2] { -rAd, 0.0 };
        double[] CutOut2Point2 = new double[2] { -1*(rAd + deltaNonDim ), tubeHeight };

        var CutOut1 = new BoSSS.Platform.Utils.Geom.BoundingBox(2);
        CutOut1.AddPoint(CutOut1Point1);
        CutOut1.AddPoint(CutOut1Point2);

        var CutOut2 = new BoSSS.Platform.Utils.Geom.BoundingBox(2);
        CutOut2.AddPoint(CutOut2Point1);
        CutOut2.AddPoint(CutOut2Point2);
        var CutOuts = new BoSSS.Platform.Utils.Geom.BoundingBox[] { CutOut1, CutOut2 };

        var grd = Grid2D.Cartesian2DGrid(_xNodes, _yNodes3, CutOuts: CutOuts);
        grd.EdgeTagNames.Add(1, "Velocity_Inlet_O2");
        grd.EdgeTagNames.Add(2, "Velocity_Inlet_CH4");
        grd.EdgeTagNames.Add(3, "Pressure_Outlet");
        // grd.EdgeTagNames.Add(4, "wall");
        grd.EdgeTagNames.Add(4, "NoSlipNeumann");
        // grd.EdgeTagNames.Add(4, "Velocity_Inlet_outer"); // We want a constant velocity field.

        grd.DefineEdgeTags(delegate (double[] X) {
            double x = X[0];
            double y = X[1];

            //Edge tags
            //1: Velocity inlet O_2
            //2: Velocity inlet CH_4
            //3: Pressure outlet
            //4: NoSlipNeumann
            if(Math.Abs(x -  list2.Last() ) < 1e-8 || Math.Abs(x- list2.First() ) < 1e-8 )
            return 3;
            
            if(Math.Abs(y -(listY.ToList()).Last()) < 1e-8 )
            return 3;

            if (Math.Abs(y - ymin) < 1e-8) { // Bottom part
                if (Math.Abs(x - 0.0) < rAd + 1e-8)
                    return 2; // Velocity Inlet CH4
                if (x > wallNodes.Last() + 1e-8 || x < wallNodesLeft[0] + 1e-8) {
                    return 1;
                } 
            }

        
            return 4;

        }
        );
        
                // var gDat = new GridData(grd);
                // var em1 = gDat.GetBoundaryEdges();
                // em1.SaveToTextFile("alledges.csv", false, (double[] CoordGlobal, int LogicalItemIndex, int GeomItemIndex) => (double)gDat.iGeomEdges.EdgeTags[GeomItemIndex]);
        myDb.SaveGrid(ref grd);

        return grd;
        
    }
  
}

In [ ]:
// var grd = GridFactory.GenerateGrid_FullSystem(10);

In [ ]:
public static class BoundaryValueFactory { 

    public static string GetPrefixCode(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma) {
        using(var stw = new System.IO.StringWriter()) {

            stw.WriteLine("static class BoundaryValues {");

            stw.WriteLine("  static public double ConstantValue(double[] X) {");
            stw.WriteLine("    return "+ ConstVal +";");
            stw.WriteLine("  }");

            stw.WriteLine("  static public double ParabolaVelocityFuel(double[] X) {");
            stw.WriteLine("    return  (1.0 - Math.Pow(X[0] / "+inletRadius+", 2)) * "+uInFuel+" ;");
            stw.WriteLine("  }");

            stw.WriteLine("  static public double ParabolaVelocityFuelOscilatory(double[] X, double t) {");
            stw.WriteLine("    return  (1.0 + 3.7*Math.Sin(3.14*t) )*(1.0 - Math.Pow(X[0] / "+inletRadius+", 2)  ) * "+uInFuel+" ;");
            stw.WriteLine("  }");

            stw.WriteLine("  static public double ParabolaVelocityFuelOscilatory_tref(double[] X, double t) {");
            stw.WriteLine(" double t2 = t/"+sigma+";"); // sigma is the reference time, Lref/uref
            stw.WriteLine("    return  (1.0 + 0.7*Math.Sin(3.14*t2) )*(1.0 - Math.Pow(X[0] / "+inletRadius+", 2)  ) * "+uInFuel+" ;");
            stw.WriteLine("  }");





            stw.WriteLine("}"); 
            return stw.ToString();
        }
    }

    static public Formula Get_ConstantValue(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.ConstantValue", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }

    static public Formula Get_ParabolaVelocityFuel(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.ParabolaVelocityFuel", AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }

    static public Formula Get_ParabolaVelocityFuelOscilatory(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.ParabolaVelocityFuelOscilatory",TimeDep:true, AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }
    static public Formula Get_ParabolaVelocityFuelOscilatory_tref(double ConstVal, double inletRadius, double uInFuel, double uInAir, double sigma){
        return new Formula("BoundaryValues.ParabolaVelocityFuelOscilatory_tref",TimeDep:true, AdditionalPrefixCode:GetPrefixCode(ConstVal, inletRadius, uInFuel, uInAir,sigma));
    }
}

## Create base control file
In this ControlFile basic configuration of the CounterDiffusionFlame is defined.

In [ ]:
static XNSEC_Control GiveMeTheCtrlFile(int dg, int nCells, bool isMF, double massFuelIn, double massAirIn) {
    var CC = new ChemicalConstants();
    
    var C = isMF ? new XNSEC_MF_Control() : new XNSEC_Control();

    C.NumberOfChemicalSpecies = 4; 
    C.SetDGdegree(dg); //


    
    double r = 0.635 / 100 ; // Radius inner cylinder, m
    double delta = r / 5;
    double R = 2.54 / 100 * 3 + delta; // Radius outter cylinder, m
    double zlength = 20.0 / 100 * 2;// meters
    C.SetGrid(GridFactory.GenerateGrid_FullSystem(nCells, r, delta, R, zlength));  //
    C.MatParamsMode = MaterialParamsMode.Constant;  //

    // C.MatParamsMode = MaterialParamsMode.Sutherland;  //
    C.rhoOne = false; //!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    // Problem Definition
    //===================
    double TemperatureInFuel = 300;  //
    double TemperatureInOxidizer = 300; //
    double AtmPressure = 101325; // Pa
    double[] FuelInletConcentrations = new double[] { 0.2, 0.0, 0.0, 0.0, 0.8 };
    double[] OxidizerInletConcentrations = new double[] { 0.0, 0.23, 0.0, 0.0, 0.77 }; //
    double[] MWs = new double[] { CC.MW_CH4, CC.MW_O2, CC.MW_CO2, CC.MW_H2O, CC.MW_N2 };

    double mwFuel = CC.getAvgMW(MWs, FuelInletConcentrations);
    double mwAir = CC.getAvgMW(MWs, OxidizerInletConcentrations);
    double densityAirIn = AtmPressure * mwAir / (CC.R_gas * TemperatureInOxidizer * 1000); // kg / m3
    double densityFuelIn = AtmPressure * mwFuel / (CC.R_gas * TemperatureInFuel * 1000); // kg / m3.
    double uInFuel = massFuelIn / densityFuelIn*1 + 35.0/100*0 + 0.8*0; //
    double uInAir = massAirIn / densityAirIn*1 +35.0/100*0+ 0.16*0; //

    Console.WriteLine("VelocityFuel" + uInFuel);
    Console.WriteLine("VelocityAir" + uInAir);

    // Reference values
    //===================
    // Basic units to be used: Kg, m, s, mol, pa,
    double TRef = TemperatureInOxidizer;// Reference temperature  is the inlet temperature, (K)
    double pRef = AtmPressure; // Pa
    double uRef = Math.Max(uInFuel, uInAir); // m/s

    double LRef = r;

    C.GravityDirection = new double[] { 0.0, 0.0, 0.0 }; //No gravity.

    // Solver configuration
    // =======================
    C.smoothingFactor = 80;
    C.NonLinearSolver.ConvergenceCriterion = 1e-5;
    
    C.NonLinearSolver.verbose = true;
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.MaxSolverIterations = 10;
    
    C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
    C.saveperiod = 1;
    C.PenaltyViscMomentum = 1.0; 
    C.PenaltyHeatConduction = 1.0;
    C.YFuelInlet = FuelInletConcentrations[0];
    C.YOxInlet = OxidizerInletConcentrations[1];
    C.FuelInletConcentrations = FuelInletConcentrations;
    C.OxidizerInletConcentrations = OxidizerInletConcentrations;
    C.TFuelInlet = 1.0;
    C.TOxInlet = 1.0;
    C.PhysicalParameters.IncludeConvection = true;
    // Chemical related parameters
    double s = (CC.nu_O2 * CC.MW_O2) / (CC.nu_CH4 * CC.MW_CH4);
    C.phi = s * C.YFuelInlet / C.YOxInlet;
    C.zSt = 1.0 / (1.0 + C.phi);
    var MLC = new MaterialLawCombustion(300, new double[] { }, C.MatParamsMode, C.rhoOne, true, 1.0, 1, 1, C.YOxInlet, C.YFuelInlet, C.zSt, CC, 0.75);
    var ThermoProperties = new ThermodynamicalProperties();

    //==========================
    //Derived reference values
    //==========================
    C.uRef = uRef; // Reference velocity
    C.LRef = LRef; // reference length
    C.pRef = AtmPressure; // reference pressure
    C.TRef = TemperatureInFuel;// reference temperature
    C.MWRef = MLC.getAvgMW(MWs, C.OxidizerInletConcentrations); // Air mean molecular weight
    C.rhoRef = C.pRef * C.MWRef / (8.314 * C.TRef * 1000); // Kg/m3. ok ;
    C.cpRef = 1.3;//ThermoProperties.Calculate_Cp_Mixture(new double[] { 0.23, 0.77 }, new string[] { "O2", "N2" }, 300); // 1.219185317353029;// Representative value, KJ/Kg K ========> 1.31 for the one-step kinetic model
    C.muRef = MLC.getViscosityDim(300);
    C.MolarMasses = new double[] { C.CC.MW_CH4, C.CC.MW_O2, C.CC.MW_CO2, C.CC.MW_H2O, C.CC.MW_N2 };
    C.MolarMasses.ScaleV(1.0 / C.MWRef); //NonDimensionalized Molar masses
    C.T_ref_Sutherland = 300;
    double heatRelease_Ref = (C.TRef * C.cpRef);
    C.HeatRelease = C.CC.HeatReleaseMass / heatRelease_Ref;
    C.B = CC.PreExponentialFactor;

    C.StoichiometricCoefficients = new double[] { -1, -2, 1, 2, 0 };

    C.Damk = C.rhoRef * C.LRef * C.B / (C.uRef * C.MWRef);
    C.Reynolds = C.rhoRef * C.uRef * C.LRef / C.muRef;
    C.Prandtl = 0.75; 
    C.Schmidt = C.Prandtl; // Because Lewis number  is assumed as 1.0  (Le = Pr/Sc)
    // C.Lewis = new double[] { 0.97, 1.11, 1.39, 0.83, 1.0 };
    C.Lewis = new double[] { 1.0, 1.0, 1.0, 1.0, 1.0 };

    double g = 9.8; // m/s2
    C.Froude = Math.Sqrt(uRef * uRef / (C.LRef * g)); // Not used
    C.T_ref_Sutherland = 300; //////// Check this
    C.ReactionRateConstants = new double[] { C.Damk, CC.Ta / TRef, 1.0, 1.0 }; 




    //==========================
    // Initial conditions
    //==========================
    double dummy = 0;
    C.AddInitialValue(VariableNames.VelocityX, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, uInFuel / C.uRef, uInAir / C.uRef,dummy));
    C.AddInitialValue(VariableNames.VelocityY, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, uInFuel / C.uRef, uInAir / C.uRef,dummy));
    C.AddInitialValue(VariableNames.Pressure, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, uInFuel / C.uRef, uInAir / C.uRef,dummy));

    //==========================
    // Boundary conditions
    //==========================


    
    C.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
    C.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ParabolaVelocityFuel( dummy, r/C.LRef, uInFuel / uRef, uInAir / uRef, dummy));

    
    C.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
    C.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ConstantValue( uInAir / uRef, dummy, dummy,dummy, dummy));
    C.AddBoundaryValue("NoSlipNeumann");

    // C.AddBoundaryValue("Velocity_Inlet_outer", VariableNames.Velocity_d(0), BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy,dummy, dummy));
    // C.AddBoundaryValue("Velocity_Inlet_outer", VariableNames.Velocity_d(1), BoundaryValueFactory.Get_ConstantValue( uInAir / uRef, dummy, dummy,dummy, dummy));

    return C;
}

## Starting the MixtureFraction simulation
Configuration  for the simulation using the mixture fraction approach, where an infinite reaction rate is assumed. Used to find adequate starting solution for the full problem.

In [ ]:
static XNSEC_Control GiveMeTheMixtureFractionCtrlFile(int dg, int nCells, double massFuelIn, double massAirIn){
    var C_MixtureFraction = GiveMeTheCtrlFile(dg, nCells, true,massFuelIn, massAirIn  );

    
    C_MixtureFraction.physicsMode = PhysicsMode.MixtureFraction;
    C_MixtureFraction.ProjectName = "CoFlowFlame";
    string name = C_MixtureFraction.ProjectName + "P" + dg + "K" + nCells;
    C_MixtureFraction.SessionName = "FS_" + name;

    C_MixtureFraction.UseSelfMadeTemporalOperator = false;
    C_MixtureFraction.ChemicalReactionActive = false;
    C_MixtureFraction.physicsMode = PhysicsMode.MixtureFraction;
    C_MixtureFraction.NonLinearSolver.MaxSolverIterations = 30; 
    
    // Boundary and initial conditions
    double dummy = -11111111;

    C_MixtureFraction.AddInitialValue(VariableNames.MixtureFraction,BoundaryValueFactory.Get_ConstantValue(1.0,dummy,dummy , dummy, dummy));
    
    C_MixtureFraction.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MixtureFraction, BoundaryValueFactory.Get_ConstantValue(1.0,dummy,dummy , dummy, dummy));        
    C_MixtureFraction.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MixtureFraction, BoundaryValueFactory.Get_ConstantValue(0.0,dummy,dummy , dummy, dummy));
    

    
   
    int NoOfPseudoTimesteps =  3;
    C_MixtureFraction.TimesteppingMode = BoSSS.Solution.Control.AppControl._TimesteppingMode.Steady; 
    C_MixtureFraction.NoOfTimesteps = NoOfPseudoTimesteps ;


    C_MixtureFraction.AdaptiveMeshRefinement = true; 
    C_MixtureFraction.AMR_startUpSweeps = 3;
    double[] p1 = new double[2] { -3.0, 2.5 };
    double[] p2 = new double[2] { 3.0,6.0};
    var bb = new BoSSS.Platform.Utils.Geom.BoundingBox(2);
    bb.AddPoint(p1);
    bb.AddPoint(p2);
    
    var ind =  new BoSSS.Application.XNSEC.AMRInBoundingBox(p1,p2);
    ind.maxRefinementLevel = 1;
    C_MixtureFraction.activeAMRlevelIndicators.Add(ind);


    // C_MixtureFraction.activeAMRlevelIndicators.Add( new BoSSS.Application.XNSEC.AMR_onProblematicPoints(troubledPoints,C_MixtureFraction.AMR_startUpSweeps) );  
    // C_MixtureFraction.activeAMRlevelIndicators.Add(new BoSSS.Application.XNSEC.AMR_RefineAroundProblematicPoints(troubledPoints, 3, 0.2));
    C_MixtureFraction.activeAMRlevelIndicators.Add( new BoSSS.Application.XNSEC.AMR_onFlameSheet(C_MixtureFraction.zSt,3) );
    return C_MixtureFraction;
}

## Send and run jobs

In [ ]:
Type solver_MF = typeof(BoSSS.Application.XNSEC.XNSEC_MixtureFraction);
bool trefok = false; // shouldnt matter
var C_MixtureFraction = GiveMeTheMixtureFractionCtrlFile(dgMF, nCells, InitialMassFuelIn, InitialMassAirIn); 
string jobName       = C_MixtureFraction.SessionName;
Console.WriteLine(jobName);
var oneJob           = new Job(jobName, solver_MF);
oneJob.NumberOfMPIProcs = NumberOfProcMF;
oneJob.NumberOfThreads = 1;
oneJob.SetControlObject(C_MixtureFraction);
oneJob.Activate(myBatch); 


nCells: 10
n4: 1
rAd:1
tubeHeight:4


Grid Edge Tags changed.
VelocityFuel0.02426418525959607
VelocityAir0.06147806069578092
FS_CoFlowFlameP2K10
Deploying job FS_CoFlowFlameP2K10 ... 
Deploying executables and additional files ...
Deployment directory: \\hpccluster\hpccluster-scratch\gutierrez\binaries\CoFloFlame_MFComparison__-XNSEC2022Apr07_201208
copied 56 files.
   written file: control.obj
   copied 'amd64' runtime.
deployment finished.



In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate();

All jobs finished.


In [ ]:
static XNSEC_Control GiveMeTheFullCtrlFile(int dg, int nCells, ISessionInfo SessionToRestart, double massFuelIn, double massAirIn) {    
    
    var C_OneStep = GiveMeTheCtrlFile(dg, nCells, false,   massFuelIn, massAirIn);
    C_OneStep.physicsMode = PhysicsMode.Combustion;
    C_OneStep.ProjectName = "CoFlowFlame";

    string name = C_OneStep.ProjectName + "P" + dg + "K" + nCells;
    C_OneStep.SessionName = "Full_" + name;
    C_OneStep.VariableOneStepParameters = false; //////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////
    C_OneStep.UseSelfMadeTemporalOperator = false;
    C_OneStep.myThermalWallType = SIPDiffusionTemperature.ThermalWallType.Adiabatic;
    C_OneStep.Timestepper_LevelSetHandling = BoSSS.Solution.XdgTimestepping.LevelSetHandling.None;
    C_OneStep.UseMixtureFractionsForCombustionInitialization = true;

    C_OneStep.LinearSolver = new BoSSS.Solution.AdvancedSolvers.OrthoMGSchwarzConfig() {
        NoOfMultigridLevels = 5,
        //verbose = true
    };

    C_OneStep.ChemicalReactionActive = true;
    C_OneStep.AdaptiveMeshRefinement = true;
    
    C_OneStep.NonLinearSolver.MaxSolverIterations = 30;
    C_OneStep.TimesteppingMode = AppControl._TimesteppingMode.Steady;

    C_OneStep.NoOfTimesteps = 1;
    if (C_OneStep.ChemicalReactionActive) {
        C_OneStep.activeAMRlevelIndicators.Add(new AMR_onReactiveZones(2, 0.1));
        // C_OneStep.activeAMRlevelIndicators.Add(new AMR_BasedOnVariableLimits("Temperature", new double[] { -100, 4 },3)); // Refine all cells with T > 5 (and T < -100)
        // C_OneStep.activeAMRlevelIndicators.Add(new AMR_BasedOnFieldGradient(2, 0.1, VariableNames.Temperature));
        // C_OneStep.activeAMRlevelIndicators.Add(new AMR_BasedOnPerssonSensor(VariableNames.Temperature, 2));
    }
    // C_OneStep.activeAMRlevelIndicators.Add(new AMR_BasedOnPerssonSensor(VariableNames.Temperature, 3));
    // C_OneStep.NonLinearSolver.MaxSolverIterations = 10;

    // limiting of variable values
    Dictionary<string, Tuple<double, double>> Bounds = new Dictionary<string, Tuple<double, double>>();
    double eps = 1e-2;
    Bounds.Add(VariableNames.Temperature, new Tuple<double, double>(1.0 - eps, 10)); // Min temp should be the inlet temperature.
    Bounds.Add(VariableNames.MassFraction0, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4)); // Between 0 and 1 per definition
    Bounds.Add(VariableNames.MassFraction1, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4));
    Bounds.Add(VariableNames.MassFraction2, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4));
    Bounds.Add(VariableNames.MassFraction3, new Tuple<double, double>(0.0 - 1e-4, 1.0 + 1e-4));
    C_OneStep.VariableBounds = Bounds;
    // Boundary conditions

    double dummy = 0;
    if (SessionToRestart != null) {
        C_OneStep.SetRestart(SessionToRestart);
    } else {
        C_OneStep.AddInitialValue(VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantValue(0.23, dummy, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
        C_OneStep.AddInitialValue(VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantValue(0.0, dummy, dummy, dummy, dummy));
    }

    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[0], dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[1], dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[2], dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_CH4", VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantValue(C_OneStep.FuelInletConcentrations[3], dummy, dummy, dummy, dummy));

    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.Temperature, BoundaryValueFactory.Get_ConstantValue(1.0, dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction0, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[0], dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction1, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[1], dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction2, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[2], dummy, dummy, dummy, dummy));
    C_OneStep.AddBoundaryValue("Velocity_Inlet_O2", VariableNames.MassFraction3, BoundaryValueFactory.Get_ConstantValue(C_OneStep.OxidizerInletConcentrations[3], dummy, dummy, dummy, dummy));

    return C_OneStep;
}

In [ ]:
Type solver = typeof(BoSSS.Application.XNSEC.XNSEC);

In [ ]:
var sess = (myDb.Sessions.Where(s => s.Name == "FS_CoFlowFlameP" + dgMF + "K" + nCells)).FirstOrDefault();
if(sess != null){
Console.WriteLine("Restarting from session with name " + sess.Name); 
} else{
Console.WriteLine("No session found for restart. Starting simulation without combustion");
}
var C = GiveMeTheFullCtrlFile(dgFull, nCells, sess,  InitialMassFuelIn, InitialMassAirIn); 
string jobName = C.SessionName;
Console.WriteLine(jobName);
var oneJob = new Job(jobName, solver);
oneJob.NumberOfMPIProcs = 12;
oneJob.NumberOfThreads = 1;
oneJob.SetControlObject(C);
oneJob.Activate(myBatch);

Restarting from session with name FS_CoFlowFlameP2K10
nCells: 10
n4: 1
rAd:1
tubeHeight:4


Grid Edge Tags changed.
An equivalent grid (79150394-54fd-4da6-bb16-0bc641a59f91) is already present in the database -- the grid will not be saved.
VelocityFuel0.02426418525959607
VelocityAir0.06147806069578092
Full_CoFlowFlameP3K10
Deploying job Full_CoFlowFlameP3K10 ... 
Deploying executables and additional files ...
Deployment directory: \\hpccluster\hpccluster-scratch\gutierrez\binaries\CoFloFlame_MFComparison__-XNSEC2022Apr07_201949
copied 56 files.
   written file: control.obj
   copied 'amd64' runtime.
deployment finished.



## Plot temperature profiles along different y values 

In [ ]:
// double[] yVals = new double[] { 5.0, 8.0, 11, 14};
// double[] xCoords = GenericBlas.Linspace(-6.0,+6.0,61);
double[] yVals = new double[] { 5.0,10,15,20,25,30,35,40};
double[] xCoords = GenericBlas.Linspace(-6.0,+6.0,50);

In [ ]:


var TemperatureFieldFull = myDb.Sessions[0].Timesteps.Last().Fields.Where(f=>f.Identification == "Temperature").SingleOrDefault();
var TemperatureFieldMixtFract = myDb.Sessions[0].Timesteps.First().Fields.Where(f=>f.Identification == "Temperature").SingleOrDefault();





In [ ]:




List<double[]> TemperaturePicks = new List<double[]>();
List<double[]> TemperaturePicksMF = new List<double[]>();
for(int i = 0; i < yVals.Length; i++){
    double[] TemperatureSol = new double[xCoords.Length];
    double[] TemperatureSolMF = new double[xCoords.Length];

    for(int j = 0; j < xCoords.Length; j++){ // loop over X coordinate
        TemperatureSol[j] = TemperatureFieldFull.ProbeAt(new double[] {xCoords[j],yVals[i]});
        // TemperatureSolMF[j] = TemperatureFieldMixtFract.ProbeAt(new double[] {xCoords[j],yVals[i]});

    }
    
    TemperaturePicks.Add(TemperatureSol);
    // TemperaturePicksMF.Add(TemperatureSolMF);


}


In [ ]:
LineColors[] allColors = Enum.GetValues(typeof(LineColors)).Cast<LineColors>().ToArray();
PointTypes[] allPointTypes = Enum.GetValues(typeof(PointTypes)).Cast<PointTypes>().ToArray();
// PointTypes[] myPointTypes = new PointTypes[] { PointTypes.Diamond, PointTypes.Box, PointTypes.LowerTriangle, PointTypes.OpenLowerTriangle, };
// LineColors[] myCollors = new LineColors[] { LineColors.Red, LineColors.Orange, LineColors.Blue , LineColors.Green, LineColors.Yellow, LineColors.Black};


var plot = new Plot2Ddata();
for(int i = 0 ; i  < 8; i++){
    
    var fmt = new PlotFormat();
    fmt.Style = Styles.LinesPoints;
    fmt.PointType = allPointTypes[i];    

    fmt.LineColor = allColors[i];    
    plot.AddDataGroup("y = "+yVals[i],xCoords, TemperaturePicks[i],  fmt);


}


plot.PlotNow ()


Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 1 
 
 
 
 
 2 
 
 
 
 
 3 
 
 
 
 
 4 
 
 
 
 
 5 
 
 
 
 
 6 
 
 
 
 
 7 
 
 
 
 
 -6 
 
 
 
 
 -4 
 
 
 
 
 -2 
 
 
 
 
 0 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y = 5 
 
 
 y = 5 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M610.7,57.1 L664.1,57.1 M37.3,488.6 L51.8,488.6 L66.4,488.6 L80.9,488.6 L95.5,488.6 L110.0,488.6
 L124.6,488.6 L139.1,488.6 L153.7,488.6 L168.2,488.6 L182.8,488.5 L197.3,488.2 L211.9,486.9 L226.4,482.3
 L241.0,468.7 L255.5,434.6 L270.1,367.1 L284.6,252.4 L299.1,83.6 L313.7,112.8 L328.2,190.6 L342.8,269.9
 L357.3,337.3 L371.9,384.4 L386.4,408.1 L401.0,408.4 L415.5,385.2 L430.1,338.5 L444.6,271.4 L459.2,192.1
 L473.7,114.1 L488.3,81.1 L502.8,250.3 L517.3,365.9 L531.9,434.0 L546.4,468.4 L561.0,482.2 L575.5,486.9
 L590.1,488.2 L604.6,488.5 L619.2,488.6 L633.7,488.6 L648.3,488.6 L662.8,488.6 L677.4,488.6 L691.9,488.6
 L706.5,488.6 L721.0,488.6 L735.6,488.6 L750.1,488.6 '/> 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 

 
 
 y = 10 
 
 
 y = 10 
 
 
 
	<path stroke='rgb( 0, 255, 0)' d='M610.7,81.1 L664.1,81.1 M37.3,488.6 L51.8,488.5 L66.4,488.4 L80.9,488.2 L95.5,487.5 L110.0,486.1
 L124.6,483.2 L139.1,477.7 L153.7,468.0 L168.2,452.4 L182.8,429.2 L197.3,397.0 L211.9,355.0 L226.4,303.1
 L241.0,242.1 L255.5,172.9 L270.1,97.6 L284.6,65.8 L299.1,89.1 L313.7,111.9 L328.2,133.3 L342.8,152.2
 L357.3,167.5 L371.9,178.3 L386.4,183.9 L401.0,183.9 L415.5,178.5 L430.1,167.8 L444.6,152.7 L459.2,133.9
 L473.7,112.5 L488.3,89.7 L502.8,66.4 L517.3,95.6 L531.9,171.1 L546.4,240.4 L561.0,301.7 L575.5,353.8
 L590.1,396.0 L604.6,428.5 L619.2,451.9 L633.7,467.7 L648.3,477.5 L662.8,483.1 L677.4,486.1 L691.9,487.5
 L706.5,488.1 L721.0,488.4 L735.6,488.5 L750.1,488.6 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y = 15 
 
 
 y = 15 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M610.7,105.1 L664.1,105.1 M37.3,486.2 L51.8,484.3 L66.4,481.0 L80.9,475.9 L95.5,468.4 L110.0,457.6
 L124.6,443.0 L139.1,424.1 L153.7,400.5 L168.2,372.3 L182.8,339.5 L197.3,302.6 L211.9,262.2 L226.4,219.1
 L241.0,174.2 L255.5,128.5 L270.1,81.5 L284.6,59.9 L299.1,72.0 L313.7,82.9 L328.2,92.7 L342.8,101.0
 L357.3,107.4 L371.9,111.8 L386.4,114.1 L401.0,114.1 L415.5,112.0 L430.1,107.6 L444.6,101.2 L459.2,93.1
 L473.7,83.3 L488.3,72.4 L502.8,60.3 L517.3,80.2 L531.9,127.0 L546.4,172.6 L561.0,217.6 L575.5,260.8
 L590.1,301.3 L604.6,338.3 L619.2,371.2 L633.7,399.7 L648.3,423.4 L662.8,442.4 L677.4,457.2 L691.9,468.1
 L706.5,475.7 L721.0,480.9 L735.6,484.2 L750.1,486.2 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y = 20 
 
 
 y = 20 
 
 
 
	<path stroke='rgb(255, 0, 255)' d='M610.7,129.1 L664.1,129.1 M37.3,474.8 L51.8,468.4 L66.4,459.9 L80.9,448.9 L95.5,435.3 L110.0,418.9
 L124.6,399.6 L139.1,377.4 L153.7,352.7 L168.2,325.6 L182.8,296.6 L197.3,266.1 L211.9,234.6 L226.4,202.7
 L241.0,171.0 L255.5,139.8 L270.1,110.2 L284.6,82.1 L299.1,59.0 L313.7,60.4 L328.2,67.0 L342.8,71.7
 L357.3,75.3 L371.9,77.8 L386.4,79.1 L401.0,79.1 L415.5,77.9 L430.1,75.4 L444.6,71.8 L459.2,67.3
 L473.7,60.6 L488.3,58.6 L502.8,80.7 L517.3,108.7 L531.9,138.4 L546.4,169.5 L561.0,201.2 L575.5,233.2
 L590.1,264.7 L604.6,295.3 L619.2,324.4 L633.7,351.6 L648.3,376.4 L662.8,398.7 L677.4,418.1 L691.9,434.7
 L706.5,448.4 L721.0,459.5 L735.6,468.1 L750.1,474.6 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


In [ ]:
myDb.Sessions[0].Export().Do()

Starting export process... Data will be written to the directory: C:\Users\gutierrez\AppData\Local\BoSSS\plots\sessions\CoFloFlame_MFComparison____Full_CoFlowFlameP3K10__c6962d05-e6e1-4cf3-be84-bdc42eadecdc


C:\Users\gutierrez\AppData\Local\BoSSS\plots\sessions\CoFloFlame_MFComparison____Full_CoFlowFlameP3K10__c6962d05-e6e1-4cf3-be84-bdc42eadecdc

In [ ]:
allPointTypes

index,value
0,Dot
1,Plus
2,Times
3,Asterisk
4,OpenBox
5,Box
6,OpenCircle
7,Circle
8,OpenLowerTriangle
9,LowerTriangle
